In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import emoji
import re
from typing import Any
from textblob import TextBlob
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

In [5]:

data = pd.read_excel('data oktober-des 2022 etilang tanpa lang dan etle.xlsx')
df = pd.DataFrame(data)
df.head()

,text_id
0,"Kembangkan ETLE, 7 Polda Dapat Penghargaan di ..."
1,Sosialisasi etle dan app smart city\n\n#polsek...
2,"Berita populer kumparanOTO, Selasa (13/12) det..."
3,"Mimin kasih info ya gaes, kita mau uji coba ET..."
4,personil lalu lintas polres badung melaksanaka...


In [37]:
def remove_enter(val)->str:
    return ' '.join(val.split())

def remove_emoji():
    emojis = sorted(emoji.EMOJI_DATA, key=len, reverse=True)
    pattern = u'(' + u'|'.join(re.escape(u) for u in emojis) + u')'
    return re.compile(pattern)

begone_emoji = remove_emoji()

def preprocessing_data(val) -> str:
    result = val.lower().strip()
    result = re.sub(r'(@|https?)\S+|#[A-Za-z0-9_]+', '', result).replace("&amp;", "dan")
    result = re.sub(r'RT[\s]+','',result)
    result = re.sub('[^.,a-zA-Z0-9 \n\.]', '', result)
    result = re.sub(r'\d', '', result)
    result = re.sub(r'\.+', " . ", result)
    result = re.sub(r'\,+', " , ", result)
    result = remove_enter(result)
    result = begone_emoji.sub(repl='', string=result)
    return result

factory = StopWordRemoverFactory()
stopwords = factory.get_stop_words()
stopword = factory.create_stop_word_remover()

def remove_stopword(val):
    removed = stopword.remove(val)
    return removed
stemmer = StemmerFactory().create_stemmer()
def stemming(val):
    hasil = stemmer.stem(val)
    return hasil

def polarity_decider(val):
    analize = TextBlob(val)
    processed = analize.translate(from_lang='id', to='en')
    if processed.polarity > 0:
        sentimen = 1
    elif processed.polarity < 0:
        sentimen = -1
    else:
        sentimen = 0
    return sentimen

def translate(val):
    analize = TextBlob(val)
    processed = analize.translate(from_lang='id', to='en')
    return str(processed)


In [4]:
from langdetect import detect
text = df[0][0]
cleaned = preprocessing_data(text)
test = detect(cleaned)
print(test)
print(cleaned)


id
kembangkan etle , polda dapat penghargaan di rakernis fungsi gakkum


In [54]:
import csv
with open('data oktober-des 2022 etilang tanpa lang dan etle.csv', encoding='utf-8') as csv_data:
    load_data = list(csv.reader(csv_data, delimiter=';'))
    dictionary = {}
    list_of_dictionary = []
    for item in load_data[1:]:
        dictionary = {
            'text_id': preprocessing_data(item[0]),
            'text_en': translate(preprocessing_data(item[0])).lower(),
            'label' : polarity_decider(preprocessing_data(item[0]))
        }
        list_of_dictionary.append(dictionary)
    fields_names = ['text_id', 'text_en','label']
    with open('clear_dataset.csv', 'w') as csvfile:
        write = csv.DictWriter(csvfile, fieldnames= fields_names)
        write.writeheader()
        write.writerows(list_of_dictionary)


NotTranslated: Translation API returned the input string unchanged.

In [50]:
list_of_dictionary

[{'text_id': 'kembangkan etle , polda dapat penghargaan di rakernis fungsi gakkum',
  'text_en': 'develop ethle, polda can be awarded at the rakernis function gakkum',
  'label': 0},
 {'text_id': 'sosialisasi etle dan app smart city',
  'text_en': 'etle socialization and app smart city',
  'label': 1},
 {'text_id': 'berita populer kumparanoto , selasa detail vespa batik seharga rp juta , cara kerja dan jenis pelanggaran tilang etle mobile .',
  'text_en': 'popular news kumparanoto, tuesday vespa batik details for rp',
  'label': 1},
 {'text_id': 'mimin kasih info ya gaes , kita mau uji coba etle di wilayah tangerang kota nih . yukss patuhi peraturan lalu lintas , biar ngga dapet surat cinta dari mimin',
  'text_en': "mimin give me info, we want to test etle in the tangerang city area. yukss obey traffic rules, so you don't get a love letter from mimin",
  'label': 1},
 {'text_id': 'personil lalu lintas polres badung melaksanakan teguran secara humanis kepada masyarakat yang tidak mempe

AttributeError: 'str' object has no attribute 'keys'

In [24]:
for key in df['text_id']:

ValueError: Length of values (67) does not match length of index (7621)

In [ ]:
tester = {

}